IMPORT LIBRARY

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


CODES POSTAL AND BEAUTIFULSOUP

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

TABLE

In [5]:
table = soup.find('table', class_='wikitable sortable')
table_rows = table.find_all('tr')
ANS = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        ANS.append(row)

ANS


[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

DATA FRAME

In [6]:
df = pd.DataFrame(ANS, columns=["Postcode", "Borough", "Neighbourhood"])
df.head(11)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [7]:
df.tail()

,Postcode,Borough,Neighbourhood
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


In [8]:
df.shape

(289, 3)

CLEAN CELL NOT ASSIGNED

In [9]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


CELL WITH NEIGHBOURHOOD IS NOT ASSIGNED

In [10]:
d=df.index[df.Neighbourhood == 'Not assigned']
df['Neighbourhood'] = df['Borough'].where(df['Neighbourhood']=='Not assigned', df['Neighbourhood'])
df.loc[d]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


COMBINE NEIGHBOURHOOD WITH THE SAME POSTCODE

In [11]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df.shape

(103, 3)

IMPORT system operative

In [13]:
import io

COORDINATES LATITUDE AND LONGITUDE

In [14]:
http ="https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
file=requests.get(http).content
dflatitude=pd.read_csv(io.StringIO(file.decode('utf-8')))

In [15]:
dflatitude.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [16]:
dflatitude.columns = ['Postcode', 'Latitude','Longitude']

MERGE TWO DATA FRAME

In [17]:
dfmerge = pd.merge(df, dflatitude, on='Postcode')

In [18]:
dfmerge.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [68]:
# The geograpical coordinate of Toronto City are 
latitude = 43.653963
longitude = -79.387207

In [24]:
Toronto = dfmerge[dfmerge['Borough'].str.contains("Toronto")].reset_index(drop=True)
# Toronto 
print(Toronto.shape)
Toronto.head()


(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [70]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

TORONTO_map.save('torontocomplet.jpg')

In [5]:
# The geograpical coordinate of Toronto City are 
latitude = 43.728020
longitude = -79.388790

In [8]:


# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude,longitude], zoom_start=14)

map_Toronto.save('torontocomplet.html')

In [9]:
map_Toronto

In [71]:
map_Toronto

In [74]:
#  Latitude and Longtitude 
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 14)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto.save('MAPAtoronto.html')


In [ ]:
# The geograpical coordinate of Toronto City are 
latitude = 43.728020
longitude = -79.388790

In [ ]:
#manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
#manhattan_data.head()

In [25]:
# selecting only neighborhoods regarding to "Central Toronto" borough.
data = Toronto[Toronto['Borough'] == 'Central Toronto'].reset_index(drop=True)
# data = data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
data.head(9)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [26]:
Toronto.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [27]:
data.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [30]:
address_scar = 'Central Toronto, Toronto'
latitude = 43.728020
longitude = -79.388790
print('The geograpical coordinate of "Central Toronto" are: {}, {}.'.format(latitude, longitude))

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(data['Latitude'], data['Longitude'],data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Toronto)  
    
map_Toronto
map_Toronto.save('torontocenter.html')

The geograpical coordinate of "Central Toronto" are: 43.72802, -79.38879.


In [31]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [29]:
# @hiddel_cell
#CLIENT_ID = '25BQBHPZVT1D34DFCG0DLTADUWT5QZUEEEHBSIQKTYM5VAJ' # your Foursquare ID
#CLIENT_SECRET = 'U5ZLPC0X3TPQZ4ZCV00QEX3XGR1CRALGYMKWYMH00CWVC3XU' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version


In [32]:
print('Crawling different neighborhoods inside "Central Toronto"')
foursquare_dataset = foursquare_crawler(list(data['Postcode']),
                                                   list(data['Neighbourhood']),
                                                   list(data['Latitude']),
                                                   list(data['Longitude']),)

Crawling different neighborhoods inside "Central Toronto"
1.
Data is Obtained, for the Postal Code M4N (and Neighborhoods Lawrence Park) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M4P (and Neighborhoods Davisville North) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M4R (and Neighborhoods North Toronto West) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M4S (and Neighborhoods Davisville) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M4T (and Neighborhoods Moore Park, Summerhill East) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M4V (and Neighborhoods Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M5N (and Neighborhoods Roselawn) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M5P (and Neighborhoods Forest Hill North, Forest Hill West) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M5R (and Neighborhoods The Annex, North Midtown, Yorkville) SUCCESSFU

In [33]:
import pickle
with open("foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')

Received Data from Internet is Saved to Computer.


In [34]:
with open("foursquare_dataset.txt", "rb") as fp:   # Unpickling
    foursquare_dataset = pickle.load(fp)
# print(type(Scarborough_foursquare_dataset))
# Scarborough_foursquare_dataset

In [35]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [36]:
venues = get_venue_dataset(foursquare_dataset)

Number of Venuse in Coordination "M4N" Posal Code and "Lawrence Park" Negihborhood(s) is:
8
Number of Venuse in Coordination "M4P" Posal Code and "Davisville North" Negihborhood(s) is:
100
Number of Venuse in Coordination "M4R" Posal Code and "North Toronto West" Negihborhood(s) is:
43
Number of Venuse in Coordination "M4S" Posal Code and "Davisville" Negihborhood(s) is:
100
Number of Venuse in Coordination "M4T" Posal Code and "Moore Park, Summerhill East" Negihborhood(s) is:
61
Number of Venuse in Coordination "M4V" Posal Code and "Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West" Negihborhood(s) is:
76
Number of Venuse in Coordination "M5N" Posal Code and "Roselawn" Negihborhood(s) is:
22
Number of Venuse in Coordination "M5P" Posal Code and "Forest Hill North, Forest Hill West" Negihborhood(s) is:
48
Number of Venuse in Coordination "M5R" Posal Code and "The Annex, North Midtown, Yorkville" Negihborhood(s) is:
100


In [109]:
venues.head(9)

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M4N,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,This spot is popular,Park,465
1,M4N,Lawrence Park,43.728020,-79.388790,Granite Club,This spot is popular,Gym / Fitness Center,782
2,M4N,Lawrence Park,43.728020,-79.388790,Tim Hortons,This spot is popular,Coffee Shop,746
3,M4N,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,This spot is popular,Bus Line,481
4,M4N,Lawrence Park,43.728020,-79.388790,Glendon Bookstore,This spot is popular,Bookstore,797
5,M4N,Lawrence Park,43.728020,-79.388790,Glendon Forest,This spot is popular,Trail,839
6,M4N,Lawrence Park,43.728020,-79.388790,Glendon Rose Garden,This spot is popular,College Quad,853
7,M4N,Lawrence Park,43.728020,-79.388790,Glendon Athletic Club,This spot is popular,College Gym,966
8,M4P,Davisville North,43.712751,-79.390197,Sherwood Park,This spot is popular,Park,465


In [118]:
import pandas as pd
# one hot encoding
onehot = pd.get_dummies(data = venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
onehot.head(18)

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Baseball Field,Beer Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Churrascaria,Clothing Store,Coffee Shop,College Gym,College Quad,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Design Studio,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music School,Music Venue,Neighborhood,Park,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool Hall,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Shipping Store,Shoe Store,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4N,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,This spot is popular,465,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4N,Lawrence Park,43.728020,-79.388790,Granite Club,This spot is popular,782,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4N,Lawrence Park,43.728020,-79.388790,Tim Hortons,This spot is popular,746,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4N,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,This spot is popular,481,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4N,Lawrence Park,43.728020,-79.388790,Glendon Bookstore,This spot is popular,797,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,M4N,Lawrence Park,43.728020,-79.388790,Glendon Forest,This spot is popular,839,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,M4N,Lawrence Park,43.728020,-79.388790,Glendon Rose Garden,This spot is popular,853,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [119]:
print(type(venues['Venue Category']))
print(type(venues[['Venue Category']]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [125]:
onehot.columns

Index(['Postal Code', 'Neighborhood', 'Venue', 'Venue Summary', 'Distance',
       'American Restaurant', 'Arcade', 'Arts & Crafts Store',
       'Asian Restaurant', 'BBQ Joint',
       ...
       'Theater', 'Toy / Game Store', 'Track', 'Trail',
       'Vegetarian / Vegan Restaurant', 'Video Store', 'Vietnamese Restaurant',
       'Wine Bar', 'Wings Joint', 'Yoga Studio'],
      dtype='object', length=131)

DISTANCE FROM THE BUSINESS

In [132]:
a = onehot[['Distance']]
a

,Distance
0,465
1,782
2,746
3,481
4,797
5,839
6,853
7,966
8,465
9,374


CLOUSTER USING THE DISTANCE FROM BUSINESS

In [133]:
from sklearn.cluster import KMeans
kmedias=KMeans(n_clusters = 5)
kmedias.fit(a)
print(kmedias.predict(a))

[4 0 0 4 0 2 2 2 4 1 4 4 0 0 0 0 0 0 0 0 0 4 0 0 4 2 0 2 2 2 0 4 4 0 0 2 2
 2 0 0 0 0 2 0 0 0 0 0 3 2 0 0 0 0 4 2 0 0 4 0 2 0 0 0 0 0 0 2 0 0 2 0 2 0
 0 0 0 2 2 2 2 2 0 2 2 1 0 2 2 4 2 4 2 2 4 2 2 4 2 2 3 0 2 2 0 2 2 2 4 4 0
 0 4 0 4 0 0 0 0 0 0 2 4 4 4 4 1 0 4 2 0 1 4 4 2 4 4 4 2 0 0 0 2 0 2 2 2 1
 4 4 0 3 3 3 3 3 1 4 4 4 1 4 3 3 3 1 0 4 0 1 3 0 0 1 0 3 3 0 1 0 1 4 0 0 0
 0 0 2 2 0 1 2 0 2 0 2 0 0 0 4 0 2 0 0 0 0 0 4 0 0 0 2 0 1 0 2 2 2 2 2 0 2
 1 2 0 0 4 2 4 2 2 0 2 2 2 0 0 0 1 4 2 2 2 2 4 3 2 0 0 0 2 0 0 4 4 2 2 2 4
 2 2 2 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 0 2 2 0 2 2 0 2 2 2
 2 2 2 2 2 4 0 2 2 2 2 2 2 2 2 2 3 4 4 4 4 0 4 0 3 2 1 2 4 0 4 4 3 0 4 4 2
 4 4 2 4 4 0 2 2 4 4 4 4 4 4 4 4 4 0 4 2 4 4 0 4 4 0 4 1 0 4 1 4 4 4 0 4 0
 2 4 4 4 4 4 4 0 4 4 4 4 4 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 0 0 0 0 0 0 2 2 2 0 0 2 2 2 0 2 2 2 0 0 2 2 0 2 2 2 1 2 2 2 0 0 0
 0 0 0 0 0 0 0 2 2 2 2 2 2 2 1 1 1 1 1 1 4 1 1 4 0 4 1 0 4 1 4 4 4 0 1 4 0
 4 0 0 0 4 0 0 0 2 0 2 0 